In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
lyrics = pd.read_csv("/kaggle/input/bad-bunny-lyrics/bad_bunny_lyrics.csv")

In [4]:
# Good regex resource: https://realpython.com/regex-python/#python-regex-metacharacters
import re

In [15]:
practice_str = '[Letra de "NI BIEN NI MAL"]  [Intro: Bad Bunny] Yeh-yeh Yeh-yeh Yeh-yeh'
practice_song_lyric = lyrics.iloc[0,4]
practice_song_lyric

In [25]:
# figure out how to escape the square brackets
# "[\[.\]]"
re.search(r'[^\[\w\s":\]]',practice_str)

In [26]:
# Source: https://stackoverflow.com/questions/65022050/cleaning-song-lyrics-with-regex
# removes square brackets and everything between them
re.sub(r"[\[].*?[\]]", "", practice_str)

In [27]:
# re.search(r"[\[].*?[\]]",'[Letra de "NI BIEN NI MAL"]  [Intro: Bad Bunny] Yeh-yeh Yeh-yeh Yeh-yeh  [Coro: Bad Bunny] Sin ti no me va bien, tampoco me va mal (Yeah) Pase lo que pase')

In [43]:
removed_brackets = re.sub(r"[\[].*?[\]]","",practice_song_lyric)
no_brackets_or_new_lines = re.sub(r"[\n]"," ",removed_brackets)
no_brackets_or_new_lines

In [65]:
# might need to replace new lines with periods
correct_vas = re.sub(r"va\'","vas",no_brackets_or_new_lines)
correct_para_zero = re.sub(r"Pa\'l","Para el",correct_vas)
correct_para = re.sub(r"Pa\'","Para ",correct_para_zero)
correct_voy = re.sub(r"vo\'","voy ",correct_para)
correct_para_2 = re.sub(r"pa\'","para",correct_voy)
correct_de = re.sub(r"\'e","de",correct_para_2)
correct_puyas = re.sub(r"puya\'","puyas",correct_de)
correct_puyas

Spanish Sentiment Analyis [Library](https://pypi.org/project/sentiment-analysis-spanish/) (uses CNN).

In [ ]:
# replaces all Unicode whitespace (such as \u) with a single space
#re.sub("\s+", " ", lyric, flags=re.UNICODE)

In [ ]:
# Source: https://t-redactyl.io/blog/2017/06/text-cleaning-in-multiple-languages.html
# from nltk.stem.snowball import SnowballStemmer

In [ ]:
# sbEsp = SnowballStemmer('spanish')

In [ ]:
import spacy

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
# load a Spanish-language Spacy model
import es_core_news_sm
nlp = es_core_news_sm.load()

In [ ]:
# Spanish lemmatization (pattern)
# Spanish word normalization Python packages and libraries (Cucco)
# ftfy - handles mojibake
# Tokenize - nltk package (tokenize and FreqDist)